In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 14s 16ms/step - loss: 3.5914 - acc: 0.0289 - val_loss: 3.5851 - val_acc: 0.0093
Epoch 2/20
864/864 [==============================] - 7s 8ms/step - loss: 3.5638 - acc: 0.0359 - val_loss: 3.5124 - val_acc: 0.0185
Epoch 3/20
864/864 [==============================] - 6s 7ms/step - loss: 3.2033 - acc: 0.1238 - val_loss: 3.0333 - val_acc: 0.1806
Epoch 4/20
864/864 [==============================] - 6s 7ms/step - loss: 2.4373 - acc: 0.3067 - val_loss: 2.3959 - val_acc: 0.3796
Epoch 5/20
864/864 [==============================] - 6s 7ms/step - loss: 1.7187 - acc: 0.5046 - val_loss: 1.9145 - val_acc: 0.4676
Epoch 6/20
864/864 [==============================] - 6s 7ms/step - loss: 1.1481 - acc: 0.6377 - val_loss: 1.6720 - val_acc: 0.5602
Epoch 7/20
864/864 [==============================] - 7s 8ms/step - loss: 0.7412 - acc: 0.7847 - val_loss: 1.5659 - val_acc: 0.6065
Epoch 8/20
864/864 [========

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('test/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['3', '4', 'C', 'E', 'e', 'Y', 'h', 'X', 'K', '6', 'S', 'y', '8', 'n', '7', 'Z', 'q', 'R', 'B', 'm', 'A', 'b', 'U', 'G', 'M', 'W', 'f', 'a', 'd', '5', 'N', 'F', 'P', '2', 'Q', 'H', 'V']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


In [7]:
#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

In [8]:
#print(loss_value, metrics_value)

In [9]:
import random

In [10]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  WyhU7
Predicted: WyhU7 

Elapsed time=  0.20079633831185717 s
<------------------------------->


Actual:  UdYCH
Predicted: UdYCH 

Elapsed time=  0.8708819171013222 s
<------------------------------->


Actual:  KCPne
Predicted: KCPne 

Elapsed time=  0.28062694686588396 s
<------------------------------->


Actual:  KBhVF
Predicted: KBhVF 

Elapsed time=  0.5685595525365786 s
<------------------------------->


Actual:  XUVfh
Predicted: XUVfh 

Elapsed time=  0.47090541366845684 s
<------------------------------->


Actual:  qm4XQ
Predicted: EeQM 

Elapsed time=  0.7956212066638249 s
<------------------------------->


Actual:  aV5Fa
Predicted: aV5Fa 

Elapsed time=  0.8780301402251443 s
<------------------------------->


Actual:  C7y3B
Predicted: G2Eb 

Elapsed time=  0.8240975068047522 s
<------------------------------->


Actual:  X7EUC
Predicted: X7EUC 

Elapsed time=  0.4238905878549193 s
<------------------------------->


Actual:  AZfq4
Predicted: eHNb 

Elapsed ti

Predicted: hZPZ 

Elapsed time=  0.7486081812293143 s
<------------------------------->


Actual:  3PN3a
Predicted: 3PN3a 

Elapsed time=  0.2591112135044602 s
<------------------------------->


Actual:  ZeKmV
Predicted: ZeKmV 

Elapsed time=  0.42791432463621903 s
<------------------------------->


Actual:  eRXMP
Predicted: eRXMP 

Elapsed time=  0.9967870498493134 s
<------------------------------->


Actual:  qVCXK
Predicted: qVCXK 

Elapsed time=  0.45828044687477665 s
<------------------------------->


Actual:  EKXUP
Predicted: EKXUP 

Elapsed time=  0.27892836405724986 s
<------------------------------->


Actual:  fEXeP
Predicted: fEXeP 

Elapsed time=  0.7905142590605614 s
<------------------------------->


Actual:  7hmZC
Predicted: 7hmZC 

Elapsed time=  0.7704151560587075 s
<------------------------------->


Actual:  4QZCE
Predicted: 4QZCE 

Elapsed time=  0.49446546137283076 s
<------------------------------->


Actual:  PahKF
Predicted: Fne2 

Elapsed time=  0.42196052

Predicted: hFQWS 

Elapsed time=  0.7760791417081847 s
<------------------------------->


Actual:  GSY8a
Predicted: GSY8a 

Elapsed time=  0.49964488846197097 s
<------------------------------->


Actual:  EMf66
Predicted: EMf66 

Elapsed time=  0.0686913041530478 s
<------------------------------->


Actual:  CVXCH
Predicted: CVXCH 

Elapsed time=  0.26487245402927106 s
<------------------------------->


Actual:  5PSmh
Predicted: 5PSmh 

Elapsed time=  0.5273311649332498 s
<------------------------------->


Actual:  qSPGC
Predicted: qSPGC 

Elapsed time=  0.6464240085301076 s
<------------------------------->


Actual:  hUV5W
Predicted: hUV5W 

Elapsed time=  0.9730024073933168 s
<------------------------------->


Actual:  EbUhW
Predicted: EbUhW 

Elapsed time=  0.3976701978232319 s
<------------------------------->


Actual:  dAEXU
Predicted: dAEXU 

Elapsed time=  0.9140722293254469 s
<------------------------------->


Actual:  qMn8U
Predicted: qMn8U 

Elapsed time=  0.56684006

Predicted: nZeeb 

Elapsed time=  0.32153326549514627 s
<------------------------------->


Actual:  XdK2m
Predicted: XdK2m 

Elapsed time=  0.13449603484288053 s
<------------------------------->


Actual:  KUCfV
Predicted: 3d3h 

Elapsed time=  0.2042587212625102 s
<------------------------------->


Actual:  ZSaP6
Predicted: ZSaP6 

Elapsed time=  0.7548587751971126 s
<------------------------------->


Actual:  W5fX2
Predicted: W5fX2 

Elapsed time=  0.5457516687937964 s
<------------------------------->


Actual:  UdYCH
Predicted: UdYCH 

Elapsed time=  0.032100533305389645 s
<------------------------------->


Actual:  Z74UV
Predicted: ZEen 

Elapsed time=  0.25052840790789266 s
<------------------------------->


Actual:  RBPnB
Predicted: RBPnB 

Elapsed time=  0.27284685523957053 s
<------------------------------->


Actual:  UHHm7
Predicted: UHHm7 

Elapsed time=  0.7920979010919621 s
<------------------------------->


Actual:  CbZqA
Predicted: CbZqA 

Elapsed time=  0.379636

Predicted: m5YM 

Elapsed time=  0.9984152998776573 s
<------------------------------->


Actual:  CUUhf
Predicted: nZ8h 

Elapsed time=  0.5660174734124382 s
<------------------------------->
Total number of CAPTCHAS broken:  381
Total number of CAPTCHAS attempted:  500
Total time taken=  161.33748312897853 s
Accuracy=  76.2
